<a href="https://colab.research.google.com/github/Chandralekha005/machine-learning/blob/main/Pneumonia_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
import os
from google.colab import files

In [ ]:
if not os.path.exists("kaggle.json"):
    print("Please upload your kaggle.json file")
    files.upload()

Please upload your kaggle.json file


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia -p ./chest_xray_pneumonia --unzip

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100% 2.29G/2.29G [01:18<00:00, 32.5MB/s]
100% 2.29G/2.29G [01:18<00:00, 31.4MB/s]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 224, 224 # Image dimensions for resizing
BATCH_SIZE = 32 # Number of images per batch
EPOCHS = 4 # Set number of epochs (to fit within your time frame)
DATASET_PATH = './chest_xray_pneumonia/chest_xray'

In [ ]:
train_data_gen = ImageDataGenerator(
    rescale=1.0/255, # Normalize pixel values to [0, 1]
    shear_range=0.2, # Shear transformation for augmentation
    zoom_range=0.2, # Zoom transformation for augmentation
    horizontal_flip=True, # Flip images horizontally for augmentation
    validation_split=0.2 # 80-20 train-validation split
)

In [ ]:
train_generator = train_data_gen.flow_from_directory(
    DATASET_PATH + '/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary', # Binary classification (Pneumonia vs Normal)
    subset='training' # Use the training subset
)

Found 4173 images belonging to 2 classes.


In [ ]:
validation_generator = train_data_gen.flow_from_directory(
    DATASET_PATH + '/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation' # Use the validation subset
)

Found 1043 images belonging to 2 classes.


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(), # Flatten the 2D outputs to 1D
    Dense(128, activation='relu'), # Fully connected layer
    Dropout(0.5), # Dropout for regularization
    Dense(1, activation='sigmoid') # Sigmoid output for binary classification
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer=Adam(),
              loss='binary_crossentropy', # For binary classification
              metrics=['accuracy'])

# Step 8: Train the Model (with 4 epochs)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

# Save the Model
model.save('pneumonia_detection_model.h5')
print("Model saved as pneumonia_detection_model.h5")

# Step 9: Evaluate the Model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")


Epoch 1/4


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


130/130 ━━━━━━━━━━━━━━━━━━━━ 677s 5s/step - accuracy: 0.7266 - loss: 0.5792 - val_accuracy: 0.8721 - val_loss: 0.2784
Epoch 2/4
  1/130 ━━━━━━━━━━━━━━━━━━━━ 9:37 4s/step - accuracy: 0.8750 - loss: 0.3404

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8750 - loss: 0.3404 - val_accuracy: 0.9474 - val_loss: 0.1530
Epoch 3/4
130/130 ━━━━━━━━━━━━━━━━━━━━ 674s 5s/step - accuracy: 0.8698 - loss: 0.3084 - val_accuracy: 0.8643 - val_loss: 0.2723
Epoch 4/4
130/130 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9062 - loss: 0.1681 - val_accuracy: 0.8421 - val_loss: 0.2818


Model saved as pneumonia_detection_model.h5
33/33 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.8819 - loss: 0.2589
Validation Loss: 0.2778964936733246
Validation Accuracy: 0.8782358765602112


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [ ]:
model = load_model('pneumonia_detection_model.h5')
print("Model loaded successfully.")

Model loaded successfully.


In [ ]:
test_data_gen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
test_generator = test_data_gen.flow_from_directory(
    DATASET_PATH + '/test', # Path to the test data
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1, # Batch size of 1 for single image predictions
    class_mode='binary', # Binary classification
    shuffle=False # Do not shuffle to keep predictions in order
)

Found 624 images belonging to 2 classes.


In [ ]:
predictions = model.predict(test_generator, steps=test_generator.samples)

624/624 ━━━━━━━━━━━━━━━━━━━━ 36s 57ms/step


In [ ]:
predicted_classes = [1 if pred > 0.5 else 0 for pred in predictions] # Assuming threshold 0.5
true_classes = test_generator.classes # True labels from the generator
class_labels = list(test_generator.class_indices.keys()) # ['NORMAL', 'PNEUMONIA']

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))


Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.91      0.50      0.65       234
   PNEUMONIA       0.76      0.97      0.86       390

    accuracy                           0.79       624
   macro avg       0.84      0.74      0.75       624
weighted avg       0.82      0.79      0.78       624



In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))


Confusion Matrix:
[[117 117]
 [ 11 379]]
